<span STYLE="font-size:150%"> 
    Convert .tif files to .nrrd
</span>

Docker image: gnasello/slicer-env:2023-10-10 \
Latest update: 10 October 2023

- load image stack in Slicer
- segment mineralized tissue
- compute segmented statistics (volumes)

# Load libraries

In [ ]:
import pyslicer as ps
import slicer
from pathlib import Path
import pandas as pd
import glob
import os

# Image input

Write the path of the image file directory and the name of the output files

In [ ]:
# this cell is tagged 'parameters'
image_directory_name = 'tif_stack'
output_dir_path = 'microCT_volume'

Get first image .tif file

In [ ]:
tif_files = [file for file in os.listdir(image_directory_name) if file.endswith(".tif") and file[-8:-4].isdigit()]
image_file_name = min(tif_files, key=lambda x: int(x[-8:-4]))

image_file_path = Path(image_directory_name) / image_file_name
image_file = str(image_file_path)
image_file

In [ ]:
path = Path(image_file)

# Remove image numbering _0000, _0001 ...
filename_output = path.stem[:-4]

Make ```segmented_volumes``` folder

In [ ]:
output_directory = Path(output_dir_path)

output_directory.mkdir(parents=True, exist_ok=True)

# Load image

## Load stack of image files as a 3D volume into 3D Slicer

Write the [X, Y, Z] voxel spaxing if you want it to be different from metadata.

If ```manual_spacing = False```, it will use default spacing value embedded in the image files

In [ ]:
def find_pca_file(pca_directory_name):
    """Find the first file with a .pca extension in the specified directory."""
    for file_name in os.listdir(pca_directory_name):
        if file_name.endswith(".pca"):
            return os.path.join(pca_directory_name, file_name)
    return None
    
def read_voxel_size(file_path):
    """Read the value of VoxelSizeX from the given file."""
    try:
        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith("VoxelSizeX"):
                    # Extract the value after '='
                    voxel_size = line.split('=')[1].strip()
                    return float(voxel_size)
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return None

In [ ]:
# Call the function and print the result
pca_file = find_pca_file(pca_directory_name)
if pca_file:
    voxel_size = read_voxel_size(pca_file)
    if voxel_size is not None:
        print(f"The value of VoxelSizeX from {pca_file} is: {voxel_size}")
    else:
        print("VoxelSizeX not found in the file.")
else:
    print("No .pca file found in the specified directory.")

In [ ]:
#manual_spacing = False
manual_spacing = [voxel_size, voxel_size, voxel_size] # mm

In [ ]:
manual_spacing

Set the quality resolution of the output volume. 

The default quality is ```'preview'``` and you get it with ```quality = Flase```

In [ ]:
quality = 'preview'
# quality = 'half'
# quality = 'full'
#quality = False

In [ ]:
masterVolumeNode = ps.load.imagestacks(first_image_file = image_file, 
                                       quality=quality,
                                       spacing=manual_spacing)

Print spacing

In [ ]:
## mm
masterVolumeNode.GetSpacing()

Export volume Node as `.nrrd` file

In [ ]:
filename_output = 'microCT_volume_preview.nrrd'

slicer.util.exportNode(masterVolumeNode, output_directory / filename_output)